In [ ]:
#%%
import plotly.graph_objects as go
import numpy as np
from ipywidgets import FloatSlider, VBox

L = 5
n_lines = 20
grid = np.linspace(-L, L, n_lines)

# Function to create horizontal and vertical lines
def create_lines(grid, resolution=100):
    dense_grid = np.linspace(grid.min(), grid.max(), resolution)
    h_lines = np.array([np.vstack((dense_grid, np.full_like(dense_grid, y))) for y in grid])
    v_lines = np.array([np.vstack((np.full_like(dense_grid, x), dense_grid)) for x in grid])
    # v_lines = np.array([np.vstack((np.full_like(dense_grid, x), dense_grid))) for x in grid]
    return h_lines, v_lines


# Generate lines
h_lines, v_lines = create_lines(grid)
h_z = h_lines[:, 0] + 1j * h_lines[:, 1]
v_z = v_lines[:, 0] + 1j * v_lines[:, 1]

def f(z, alpha):
    return z**alpha

def z_to_xy(z):
    return np.stack((np.real(z), np.imag(z)))

def xy_to_z(xy):
    return xy[0] + 1j * xy[1]

fig = go.FigureWidget()

# Create a data structure to hold line information
data = [{
    'xy': line,
    'z': xy_to_z(line),
    'color': 'red',
} for line in h_lines]
data += [{
    'xy': line,
    'z': xy_to_z(line),
    'color': 'blue',
} for line in v_lines]

# Add traces to the figure and store references to them in the data dictionary
for d in data:
    trace = go.Scatter(
        x=d['xy'][0],
        y=d['xy'][1],
        mode='lines',
        line=dict(color=d['color']),
        showlegend=False,
    )
    fig.add_trace(trace)
    d['trace'] = trace

fig.update_layout(xaxis=dict(range=[-5, 5]), yaxis=dict(range=[-5, 5]))
# Create a slider
slider = FloatSlider(min=1, max=2, step=0.01, value=1)

# Define the update function for the slider
def update_line_points(change):
    alpha = change['new']
    with fig.batch_update():
        for i, d in enumerate(data):
            x, y = z_to_xy(f(d['z'], alpha))
            fig.data[i].x = x  # Update trace.x
            fig.data[i].y = y  # Update trace.y

# Observe the slider for changes and call the update function
slider.observe(update_line_points, 'value')

# Display the plot and slider
VBox([slider, fig])